In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
DATASET_PUBMED_RCT = './gdrive/Shareddrives/DATASETS/PUBMED_RCT/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [3]:
!pip install transformers -q
!pip install sentence_transformers -q

     |████████████████████████████████| 4.7 MB 12.5 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 596 kB 69.0 MB/s 
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 1.3 MB 35.1 MB/s 


In [4]:
import pandas as pd

In [5]:
df_train = pd.read_parquet(DATASET_PUBMED_RCT + 'train.parquet')
df_dev = pd.read_parquet(DATASET_PUBMED_RCT + 'df_dev_triplets.parquet')
df_test = pd.read_parquet(DATASET_PUBMED_RCT + 'df_test_triplets.parquet')

In [6]:
df_train.columns

Index(['pmid', 'label', 'sentence', 'label_id'], dtype='object')

In [7]:
df_dev.columns

Index(['anchor', 'positive', 'negative'], dtype='object')

In [8]:
# same text but with different labels
print(f'{len(df_train.index)}')
mask = df_train.groupby('sentence')['label'].transform('nunique') > 1
df_train = df_train[~mask].copy()
print(f'{len(df_train.index)}')

180040
179892


In [9]:
from sentence_transformers import InputExample
from tqdm import tqdm

train_set = []
guid = 1
for idx, row in tqdm(df_train.iterrows(), total=len(df_train.index)):
    train_set.append(InputExample(
        guid=guid,
        texts=[row['sentence']],
        label=row['label_id']
    ))
    guid += 1
len(train_set)

100%|██████████| 179892/179892 [00:14<00:00, 12059.47it/s]


179892

In [10]:
dev_set = []

guid = 1
for idx, row in tqdm(df_dev.iterrows(), total=len(df_dev.index)):
    dev_set.append(InputExample(
        guid=guid,
        texts=[row['anchor'], row['positive'], row['negative']],
    ))
    guid += 1
len(dev_set)

100%|██████████| 30212/30212 [00:01<00:00, 19704.87it/s]


30212

In [11]:
test_set = []

guid = 1
for idx, row in tqdm(df_test.iterrows(), total=len(df_test.index)):
    test_set.append(InputExample(
        guid=guid,
        texts=[row['anchor'], row['positive'], row['negative']],
    ))
    guid += 1
len(test_set)

100%|██████████| 30122/30122 [00:01<00:00, 17670.86it/s]


30122

In [12]:
import logging
from sentence_transformers import LoggingHandler

logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [13]:
from datetime import datetime
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_file_name = model_name.split('/')[-1] + '_PubMedRCT_TripletAll'

train_batch_size = 32
output_path = (
    OUTPUT_MODEL_DIR
    + model_file_name
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
output_path

'./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12'

In [14]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer(model_name, device='cuda')

2022-08-08 13:15:12 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [15]:
from sentence_transformers import datasets
from torch.utils.data import DataLoader

loader = DataLoader(train_set, shuffle=True, batch_size=train_batch_size)

In [16]:
from sentence_transformers import losses

train_loss = losses.BatchAllTripletLoss(model=model)

In [17]:
from sentence_transformers.evaluation import TripletEvaluator

dev_evaluator = TripletEvaluator.from_input_examples(
    dev_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-dev'
)

In [18]:
logging.info("Performance before fine-tuning:")
dev_evaluator(model)

2022-08-08 13:15:26 - Performance before fine-tuning:
2022-08-08 13:15:26 - TripletEvaluator: Evaluating the model on pubmed-rct-dev dataset:


Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

2022-08-08 13:16:20 - Accuracy Cosine Distance:   	57.27
2022-08-08 13:16:20 - Accuracy Manhattan Distance:	57.21
2022-08-08 13:16:20 - Accuracy Euclidean Distance:	57.27



0.5726532503640938

In [19]:
logging.info("Evaluating model on test set")
test_evaluator = TripletEvaluator.from_input_examples(
    test_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-test'
)
test_evaluator(model)

2022-08-08 13:16:20 - Evaluating model on test set
2022-08-08 13:16:20 - TripletEvaluator: Evaluating the model on pubmed-rct-test dataset:


Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

2022-08-08 13:17:06 - Accuracy Cosine Distance:   	57.17
2022-08-08 13:17:06 - Accuracy Manhattan Distance:	57.04
2022-08-08 13:17:06 - Accuracy Euclidean Distance:	57.17



0.5717415842241551

In [20]:
num_epochs = 20

warmup_steps = int(len(loader) * num_epochs  * 0.1)  # 10% of train data

In [21]:
model_output_path = output_path + '/model'
checkpoint_output_path = output_path + '/checkpoint'

print(model_output_path)
print(checkpoint_output_path)

./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/model
./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint


In [22]:
%%time
model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_output_path,
    show_progress_bar=True,
    evaluator=dev_evaluator,
    save_best_model=True,
    checkpoint_save_total_limit=1,
    checkpoint_path=checkpoint_output_path    
)  

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5622 [00:00<?, ?it/s]

2022-08-08 13:17:44 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/500
2022-08-08 13:18:17 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/1000
2022-08-08 13:18:52 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/1500
2022-08-08 13:19:26 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/2000
2022-08-08 13:20:01 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/2500
2022-08-08 13:20:35 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/3000
2022-08-08 13:21:09 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/3500
2022-08-08 13:21:43 - Save m

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

...

2022-08-08 15:35:40 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/107000
2022-08-08 15:36:15 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/107500
2022-08-08 15:36:50 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/108000
2022-08-08 15:37:24 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/108500
2022-08-08 15:38:00 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/109000
2022-08-08 15:38:34 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/109500
2022-08-08 15:39:09 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/110000
2022-08-08 15

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

2022-08-08 15:42:49 - Accuracy Cosine Distance:   	91.36
2022-08-08 15:42:49 - Accuracy Manhattan Distance:	91.34
2022-08-08 15:42:49 - Accuracy Euclidean Distance:	91.36

2022-08-08 15:42:49 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12/checkpoint/112440
CPU times: user 2h 20min 47s, sys: 5min 34s, total: 2h 26min 22s
Wall time: 2h 25min 42s


In [23]:
logging.info("Evaluating model on test set")
test_evaluator = TripletEvaluator.from_input_examples(
    test_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-test'
)
model.evaluate(test_evaluator)

2022-08-08 15:42:49 - Evaluating model on test set
2022-08-08 15:42:49 - TripletEvaluator: Evaluating the model on pubmed-rct-test dataset:


Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

2022-08-08 15:43:38 - Accuracy Cosine Distance:   	90.19
2022-08-08 15:43:38 - Accuracy Manhattan Distance:	90.21
2022-08-08 15:43:38 - Accuracy Euclidean Distance:	90.19



0.9020981342540336

In [24]:
model.save(output_path + '_last')

2022-08-08 15:43:38 - Save model to ./gdrive/Shareddrives/MODELS/all-MiniLM-L6-v2_PubMedRCT_TripletAll-2022-08-08_13-15-12_last
